## Todo List
- [x] Checar se formato de datas conferem.
- [x] Remover UGs sem liberação OC
- [x] Considerar coluna montagem. Onde estiver vazio, pegar valor de Torres para EOL, no skate.
- [ ] Separar um cojunto aleatório para teste
- [x] Calcular tempos de construção por marco: (IO_realizado - OC_realizado), (CC_realizado - OC_realizado) ...etc  (ordem:IO,CC,ME,OT)
- [x] Calcular atraso da operação comercial (OC_realizado - OC_outorgado)
- [x] Calcular média e dp do tempo de construção.
- [ ] Calcular data dos marcos medios (OC_outorgado - tempo_medio_de_construcao) por fonte.
- [ ] Calcular atrasos com base nos marcos medios
- [ ] Calcular media e dp dos atrasos. 
- [ ] Normalizar os atrasos calculados (atraso - atraso_medio)/dp
- [ ] Construir matriz de correlação de Pearson
- [ ] Calcular os coeficientes para regressão linear
- [ ] Para UHE pegar todas UGs, para as demais pegar a primeira UG apenas 
- [ ] Comparar pegando primeira UG e media por grupo



In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
from itertools import combinations

In [2]:
def outliers(df:pd.DataFrame,cols : list):
    index_outliers =  []
    for col in cols:
        s = df[col]
        Q1 = s.quantile(0.1)
        Q3 = s.quantile(0.9)
        lower_bound =  Q1
        upper_bound =  Q3 
        index_outliers.extend(s.index[(s < lower_bound) | (s > upper_bound)])
    return set(index_outliers)

In [113]:
bd_combined = pd.read_parquet("bd_combined.gzip")
bd_combined.index.name = "idx"
bd_combined.IdeUsinaOutorga = bd_combined.IdeUsinaOutorga.astype(str)
mask = (bd_combined.SigTipoGeracao == "UHE") & (bd_combined.IdeUsinaOutorga.duplicated(keep=False))
bd_combined.loc[mask,"IdeUsinaOutorga"] = bd_combined.loc[mask,"IdeUsinaOutorga"] +"_"+ bd_combined.loc[mask].groupby("IdeUsinaOutorga").cumcount().astype(str)

In [114]:
bd_combined_OC = bd_combined[bd_combined.DatUGInicioOpComerOutorgado.notna()
                            & bd_combined.DatLiberOpComerRealizado.notna()].copy(deep=True)

In [155]:
nomes_tempo_construcao = {
    "IO_OC" : ["DatLiberOpComerRealizado","DatInicioObraRealizado"],
    "CC_OC" : ["DatLiberOpComerRealizado","DatConcretagemRealizado"],
    "ME_OC" : ["DatLiberOpComerRealizado","DatMontagemRealizado"],
    "MT_OC" : ["DatLiberOpComerRealizado","DatConclusaoTorresRealizado"],
    "OT_OC" : ["DatLiberOpComerRealizado","DatLiberacaoSFGTeste"],
    "Atraso" : ["DatLiberOpComerRealizado","DatUGInicioOpComerOutorgado"]} 

calculos_OC_por_UG = bd_combined_OC[["IdeUsinaOutorga","SigTipoGeracao"]].copy(deep=True)
for key,values in nomes_tempo_construcao.items():
    calculos_OC_por_UG[key] = bd_combined_OC[values[0]]-  bd_combined_OC[values[1]]

calculos_OC_por_UG.sample(10)

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
idx,,,,,,,,
468,29158,CGH,3134 days,2801 days,2617 days,NaT,1684 days,2435 days
3954,37584,UFV,467 days,NaT,303 days,NaT,21 days,246 days
3751,36984,EOL,326 days,295 days,159 days,27 days,32 days,-555 days
1425,31276,EOL,826 days,808 days,399 days,399 days,56 days,597 days
590,29639,UTE,497 days,NaT,NaT,NaT,33 days,146 days
4138,37953,EOL,630 days,477 days,51 days,33 days,27 days,-360 days
3472,35181,EOL,791 days,577 days,53 days,40 days,44 days,-365 days
2195,34633,EOL,507 days,469 days,9 days,9 days,44 days,-25 days
3027,33529,EOL,497 days,377 days,131 days,111 days,16 days,-1011 days


In [156]:
is_first = True 
combinacoes = list(combinations([x for x in calculos_OC_por_UG.columns if x not in ["IdeUsinaOutorga","SigTipoGeracao","Atraso"]],2))
#combinacoes = list(combinations(["IO_OC","CC_OC","ME_OC","MT_OC","OT_OC"],2))

for p_marco, s_marco in combinacoes:
    idx_loop =  ((calculos_OC_por_UG[p_marco] >= calculos_OC_por_UG[s_marco]) | 
                 calculos_OC_por_UG[p_marco].isna() | 
                 calculos_OC_por_UG[s_marco].isna())

    idx = idx_loop if is_first else (idx & idx_loop)
    is_first = False 

calculos_OC_por_UG_filtered = calculos_OC_por_UG[idx].copy(deep=True)
calculos_OC_por_UG_filtered

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
idx,,,,,,,,
0,12_0,UHE,1532 days,855 days,346 days,NaT,8 days,665 days
1,12_1,UHE,1609 days,932 days,402 days,NaT,12 days,192 days
2,19,UTE,581 days,NaT,NaT,NaT,6 days,2299 days
3,42_0,UHE,1521 days,NaT,NaT,NaT,10 days,608 days
4,42_1,UHE,1553 days,NaT,NaT,NaT,11 days,0 days
...,...,...,...,...,...,...,...,...
5752,50872,UTE,98 days,NaT,98 days,NaT,41 days,-920 days
5776,51444,UTE,99 days,NaT,99 days,NaT,22 days,-385 days
5782,51450,UTE,104 days,NaT,104 days,NaT,26 days,-380 days


In [157]:
# Atrasos por usinas são calculados com base UG com menor atraso. 
calculos_OC_por_usina = calculos_OC_por_UG.loc[calculos_OC_por_UG.groupby(["IdeUsinaOutorga"],sort=False).Atraso.idxmin()]
calculos_OC_por_usina.sample(10)

,IdeUsinaOutorga,SigTipoGeracao,IO_OC,CC_OC,ME_OC,MT_OC,OT_OC,Atraso
idx,,,,,,,,
2052,33664,EOL,289 days,230 days,108 days,108 days,17 days,-427 days
1737,31764,EOL,301 days,238 days,74 days,74 days,13 days,-384 days
1251,30806,EOL,569 days,550 days,314 days,477 days,76 days,368 days
1231,30787,EOL,572 days,448 days,319 days,314 days,36 days,238 days
2957,33020,EOL,278 days,285 days,110 days,70 days,67 days,49 days
3366,34661,EOL,231 days,141 days,78 days,26 days,43 days,117 days
2993,33430,EOL,507 days,507 days,286 days,7 days,69 days,44 days
725,29736_17,UHE,2032 days,2001 days,1911 days,NaT,34 days,-36 days
1578,31539,EOL,268 days,218 days,136 days,136 days,57 days,-21 days


In [160]:
bd_combined_OC.loc[calculos_OC_por_usina.index]

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatConclusaoTorresRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina
idx,,,,,,,,,,,,,,,,,,,,,
0,12_0,2008-12-17,2007-03-01,2008-12-17,2008-12-25,RS,100.00,NaT,2004-10-15,2005-02-01,...,NaT,NaT,2008-10-15,NaT,NaT,UHE,1,50.00,bdworgs,None
1,12_1,2009-02-28,2008-09-01,2009-02-28,2009-03-12,RS,100.00,NaT,2004-10-15,2005-02-01,...,NaT,NaT,2008-10-15,NaT,NaT,UHE,1,50.00,bdworgs,None
2,19,2001-12-03,2002-01-01,2008-04-12,2008-04-18,MG,102.89,NaT,2006-09-15,NaT,...,NaT,NaT,NaT,NaT,NaT,UTE,2,36.55,bdworgs,None
3,42_0,2005-07-20,2003-11-30,2005-07-20,2005-07-30,ES,612.10,NaT,2001-05-31,2002-01-31,...,NaT,NaT,NaT,NaT,NaT,UHE,1,102.50,bdworgs,None
4,42_1,2005-08-20,2005-08-31,2005-08-20,2005-08-31,ES,612.10,NaT,2001-05-31,2002-01-31,...,NaT,NaT,NaT,NaT,NaT,UHE,1,205.00,bdworgs,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,50872,NaT,2024-05-27,2021-10-09,2021-11-19,PR,7600.00,NaT,2021-08-13,NaT,...,NaT,NaT,NaT,NaT,2021-08-13,UTE,3,7600.00,skate,2021-05-25
5776,51444,2023-03-21,2023-04-01,2022-02-18,2022-03-12,RO,640.00,2022-05-17,2021-12-03,NaT,...,NaT,NaT,NaT,2022-10-22,2021-12-03,UTE,2,640.00,skate,2021-09-08
5782,51450,2023-03-21,2023-04-01,2022-02-19,2022-03-17,RO,640.00,2022-05-17,2021-12-03,NaT,...,NaT,NaT,NaT,2022-10-22,2021-12-03,UTE,2,640.00,skate,2021-09-08


In [135]:
# Cria tabela para calculos por geracao
calculos_por_geracao = pd.DataFrame()
calculos_por_geracao["SigTipoGeracao"] = bd_combined.SigTipoGeracao.unique() 


# Realiza calculos por geracao
for coluna in calculos_OC_por_usina.columns:
    if coluna not in ["IdeUsinaOutorga","SigTipoGeracao"]:
        calculos_por_geracao[f"{coluna}_media"] = calculos_por_geracao.SigTipoGeracao.apply(lambda x : calculos_OC_por_usina[calculos_OC_por_usina.SigTipoGeracao==x][coluna].mean())
        calculos_por_geracao[f"{coluna}_std"] = calculos_por_geracao.SigTipoGeracao.apply(lambda x : calculos_OC_por_usina[calculos_OC_por_usina.SigTipoGeracao==x][coluna].std())
display(calculos_por_geracao.set_index("SigTipoGeracao").T)

SigTipoGeracao,UHE,UTE,PCH,CGH,EOL,UFV,UTN
IO_OC_media,1629 days 14:31:24.978540768,428 days 16:26:48.121827408,633 days 15:08:29.871244632,196 days 04:09:13.846153846,557 days 01:38:24.602510464,377 days 18:40:00,NaT
IO_OC_std,1996 days 23:24:12.935696256,1641 days 20:33:11.135842832,2359 days 18:26:08.381772480,4902 days 22:43:53.915827008,371 days 06:26:50.344759416,117 days 12:52:10.536360738,NaT
CC_OC_media,1376 days 04:21:10.588235296,NaT,383 days 17:15:12.442396312,645 days 04:30:00,489 days 12:23:13.548387096,NaT,NaT
CC_OC_std,2104 days 21:40:31.381581824,NaT,2422 days 11:41:36.496784320,486 days 07:58:21.892758760,371 days 00:16:19.286303548,NaT,NaT
ME_OC_media,1157 days 09:13:50.769230768,216 days 18:43:28.049535604,139 days 02:35:29.577464788,460 days 03:16:21.818181816,300 days 12:07:05.316455696,246 days 02:02:21.176470588,NaT
ME_OC_std,2077 days 08:24:37.931928352,1768 days 07:01:59.471046400,2414 days 19:25:41.070276480,412 days 00:18:39.685175704,351 days 03:27:39.712643632,111 days 14:23:11.040855764,NaT
MT_OC_media,NaT,NaT,NaT,NaT,270 days 18:41:50.186513628,NaT,NaT
MT_OC_std,NaT,NaT,NaT,NaT,323 days 20:32:36.233120964,NaT,NaT
OT_OC_media,50 days 07:53:55.443037974,99 days 12:14:11.231527094,50 days 19:45:33.834586466,100 days 16:55:04.918032787,59 days 04:51:12.803347280,72 days 21:38:49.411764706,NaT
OT_OC_std,63 days 19:51:07.856196073,149 days 07:18:57.951231228,219 days 13:22:09.618390736,119 days 14:08:03.298001280,42 days 16:13:19.525566838,63 days 15:50:01.669435240,NaT


In [159]:
calculos_por_geracao

,SigTipoGeracao,IO_OC_media,IO_OC_std,CC_OC_media,CC_OC_std,ME_OC_media,ME_OC_std,MT_OC_media,MT_OC_std,OT_OC_media,OT_OC_std,Atraso_media,Atraso_std
0,UHE,1629 days 14:31:24.978540768,1996 days 23:24:12.935696256,1376 days 04:21:10.588235296,2104 days 21:40:31.381581824,1157 days 09:13:50.769230768,2077 days 08:24:37.931928352,NaT,NaT,50 days 07:53:55.443037974,63 days 19:51:07.856196073,201 days 15:02:14.439834024,552 days 01:30:11.559301160
1,UTE,428 days 16:26:48.121827408,1641 days 20:33:11.135842832,NaT,NaT,216 days 18:43:28.049535604,1768 days 07:01:59.471046400,NaT,NaT,99 days 12:14:11.231527094,149 days 07:18:57.951231228,245 days 03:52:08.971962616,721 days 01:02:48.732449672
2,PCH,633 days 15:08:29.871244632,2359 days 18:26:08.381772480,383 days 17:15:12.442396312,2422 days 11:41:36.496784320,139 days 02:35:29.577464788,2414 days 19:25:41.070276480,NaT,NaT,50 days 19:45:33.834586466,219 days 13:22:09.618390736,934 days 12:37:53.684210528,1148 days 13:48:30.330760768
3,CGH,196 days 04:09:13.846153846,4902 days 22:43:53.915827008,645 days 04:30:00,486 days 07:58:21.892758760,460 days 03:16:21.818181816,412 days 00:18:39.685175704,NaT,NaT,100 days 16:55:04.918032787,119 days 14:08:03.298001280,379 days 09:17:25.161290324,753 days 10:30:06.259056936
4,EOL,557 days 01:38:24.602510464,371 days 06:26:50.344759416,489 days 12:23:13.548387096,371 days 00:16:19.286303548,300 days 12:07:05.316455696,351 days 03:27:39.712643632,270 days 18:41:50.186513628,323 days 20:32:36.233120964,59 days 04:51:12.803347280,42 days 16:13:19.525566838,67 days 18:28:37.154811715,571 days 18:13:16.753657176
5,UFV,377 days 18:40:00,117 days 12:52:10.536360738,NaT,NaT,246 days 02:02:21.176470588,111 days 14:23:11.040855764,NaT,NaT,72 days 21:38:49.411764706,63 days 15:50:01.669435240,-7 days +10:49:24.705882353,291 days 00:36:17.761482092
6,UTN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [ ]:
plot_created = False

for k,origem in enumerate(["skate","bdworgs","Ambos"]):





    if not plot_created:    
        num_subplots = 6*(len(colunas_atrasos_medio)-1)
        fig, axes = plt.subplots(num_subplots,3,figsize=(24,num_subplots*5))
        plot_created =True
    count = -1
    for i,SigTipoGeracao in enumerate(calculos_por_geracao.SigTipoGeracao):
        for nome_coluna in colunas_atrasos_medio:
            if nome_coluna == "Atraso_OC_medio":
                continue
            count+=1
            try :
                ordered_notna = calculos_OC_por_usina[(calculos_OC_por_usina[nome_coluna].notna())
                                                 & (calculos_OC_por_usina.SigTipoGeracao == SigTipoGeracao) ].sort_values(by = nome_coluna)
                x = ordered_notna[nome_coluna].dt.days.to_numpy().reshape(-1, 1)/30
                y = ordered_notna["Atraso_OC_medio"].dt.days.to_numpy().reshape(-1, 1)/30
            
                res = stats.linregress(x.flatten(), y.flatten())
                a, b, r, p_value, std_err = res
                ax = axes[count,k]
                ax.scatter(x,y,color="blue")
                ax.set_xlabel(f"{nome_coluna} (meses)")
                ax.set_ylabel(f"Atraso_OC_medio (meses)")
                ax.plot(x,a*x+b+1,color="red",label= f"{a:.1f}x+{b:.1f}")
                ax.set_title(f"\nTipo de geração: {SigTipoGeracao}   -  $R^2$ = {100*r**2:.1f}%\n Número de usinas: {x.shape[0]} \nOrigem: {origem}")
                ax.legend(loc="lower right")
                ax.grid()
            except (ValueError,AttributeError) as error:
                print(rf"Não foi possível calcular: {nome_coluna} para {SigTipoGeracao}")

plt.tight_layout()
plt.savefig("calculos_atrasos.pdf")